In [2]:
import pandas as pd
import numpy as np
from sklearn.externals import joblib
from xgboost import XGBClassifier
import lightgbm as lgb

In [ ]:
import datetime
def date2weekday(date):
    date = str(date)
    year = int(date[0:4])
    month = int(date[4:6])
    day = int(date[6:])
    return datetime.datetime(year,month,day).weekday()

In [3]:
# read data
data = pd.read_csv('../data/full_size/atec_anti_fraud_train.csv',index_col=0)
data = data.sort_values(by=['date'])
train_num = int(data.shape[0]*0.8)

# generate test data
test_data = data.iloc[train_num:,:]
test_x = test_data.iloc[:,1:]
test_y = test_data.iloc[:,0]

# generate time feature/split time into month, day, weekday
df['month'] = df['date'].apply(lambda x:int(str(x)[4:6]))
df['day'] = df['date'].apply(lambda x:int(str(x)[6:]))
df['weekday'] = df['date'].apply(date2weekday)
ts_test_data = data.drop(columns=['date']).iloc[train_num:,:]
ts_test_x = ts_test_data.iloc[:,1:]
ts_test_y = test_y.copy()

In [4]:
# load models
model_root='../models/'
model_names=['xgb_300n_5d_0.1l.pkl','time_split_xgb_150n_5d_0.05l','lgb_default_early_stop.txt']
proba_matrix = {}
for model_name in model_names:
    model_path = model_root+model_name
    print('loading model {}'.format(model_name))
    if model_name.endswith('pkl'):
        clf = joblib.load(model_path)
        # predict over whole train dataset
        print('{} is predicting....'.format(model_name))
        y_predict_proba = clf.predict_proba(X)
        print("appending {}'s results...".format(model_name))
        proba_matrix[model_name] = y_predict_proba[:,1]
    elif model_name.endswith('txt'):
        clf = lgb.Booster(model_file=model_path)
        y_predict_proba = clf.predict(X)
        print("appending {}'s results...".format(model_name))
        proba_matrix[model_name] = y_predict_proba[:,1]

# build proba_matrix dataframe
pd.Dataframe(proba_matrix,index = X.index)

loading model xgb_300n_5d_0.1l.pkl
xgb_300n_5d_0.1l.pkl is predicting....
appending xgb_300n_5d_0.1l.pkl's results...
loading model lgb_default_early_stop.txt


MemoryError: 

In [ ]:
xgb_ensemble = XGBClassifier()
xgb_ensemble.fit